In [14]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
loader = TextLoader("/kaggle/input/india/india.txt")
Documents = loader.load()

In [15]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size= 500, chunk_overlap= 150)
texts = text_splitter.split_documents(Documents)

In [12]:
persist_directory = "db"
embedings = HuggingFaceEmbeddings()

<ipython-input-12-d1c08a305f01>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedings = HuggingFaceEmbeddings()
<ipython-input-12-d1c08a305f01>:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedings = HuggingFaceEmbeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedings,
                                 persist_directory=persist_directory)

In [17]:
vectordb.persist()

<ipython-input-17-d832e715b29a>:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [18]:
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedings)

<ipython-input-18-a1c60508a432>:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory,


In [19]:
from langchain.llms import HuggingFaceHub

In [20]:
retriever = vectordb.as_retriever()

In [21]:
llm = HuggingFaceHub(repo_id="meta-llama/Meta-Llama-3-8B-Instruct")

In [22]:
llm = HuggingFaceHub(repo_id="meta-llama/Meta-Llama-3-8B-Instruct")

In [23]:
llm

HuggingFaceHub(client=<InferenceClient(model='meta-llama/Meta-Llama-3-8B-Instruct', timeout=None)>, repo_id='meta-llama/Meta-Llama-3-8B-Instruct', task='text-generation')

In [24]:
from langchain.prompts import ChatPromptTemplate

In [25]:
template_1 = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much as text as possible from "response" section in the source documents.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.
Question: {question}

context : {context}

Response:"""

In [26]:
prompt=ChatPromptTemplate.from_template(template_1)

In [27]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser 

In [28]:
output_parser = StrOutputParser() 

In [29]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)   

In [30]:
query=" give me indian history in brief?"
results = rag_chain.invoke(query)
print(results)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Human: Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much as text as possible from "response" section in the source documents.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.
Question:  give me indian history in brief?

context : [Document(metadata={'source': '/kaggle/input/india/india.txt'}, page_content='Historical Background\nThe history of India is as vast and diverse as the country itself, dating back to the Indus Valley Civilization (circa 3300–1300 BCE), one of the world’s earliest urban societies. It was marked by advanced town planning, complex social structures, and trade networks extending to Mesopotamia.'), Document(metadata={'source': '/kaggle/input/india/india.txt'}, page_content='India, officially the Republic of India, is one of the oldest civilizations and a dynamic player on the global stage today. The country’s unique geographical featu

In [31]:
query_1="give me name indian prime Minister?"
print(rag_chain.invoke(query_1))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Human: Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much as text as possible from "response" section in the source documents.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.
Question: give me name indian prime Minister?

context : [Document(metadata={'source': '/kaggle/input/india/india.txt'}, page_content='Post-Independence Era\nAfter independence, India adopted a democratic system of government, with Jawaharlal Nehru becoming its first Prime Minister. The Constitution of India, enacted in 1950, established India as a sovereign, socialist, secular, and democratic republic. The constitution is the longest written constitution in the world and lays down the framework for the country’s political, legal, and social institutions.'), Document(metadata={'source': '/kaggle/input/india/india.txt'}, page_content='Economic Liberalization and Growth\nIndia’s econom

In [ ]:
import telebot
bot = telebot.TeleBot('7546573610:AAFu59llvjtCXhpT4_tb3c3ZDlyovVTNBjg')

@bot.message_handler(func=lambda message: True)
def handle_message(message):
    query = message.text
    response = rag_chain.invoke(query)
    bot.reply_to(message, response)

bot.polling() 

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_